In [2]:
import pandas as pd
import numpy as np
import sqlite3
from pybaseball import playerid_reverse_lookup
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [3]:
#Read in Data
conn = sqlite3.connect('pitching.sqlite')
df = pd.read_sql("SELECT * FROM statcast", conn)
df.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,n_thruorder_pitcher,n_priorpa_thisgame_player_at_bat,pitcher_days_since_prev_game,batter_days_since_prev_game,pitcher_days_until_next_game,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle
0,FF,2023-09-30 00:00:00,96.1,-2.08,5.57,"Smith, Drew",664761,622098,field_out,hit_into_play,...,1,4,3.0,2.0,NaN,1.0,0.85,0.55,0.55,41.2
1,FF,2023-09-30 00:00:00,96.5,-1.94,5.59,"Smith, Drew",664761,622098,None,foul,...,1,4,3.0,2.0,NaN,1.0,0.94,0.70,0.70,39.9
2,SL,2023-09-30 00:00:00,86.6,-1.91,5.54,"Smith, Drew",664761,622098,None,ball,...,1,4,3.0,2.0,NaN,1.0,3.17,-0.48,-0.48,43.8
3,SL,2023-09-30 00:00:00,87.1,-2.01,5.55,"Smith, Drew",664761,622098,None,swinging_strike,...,1,4,3.0,2.0,NaN,1.0,2.98,-0.68,-0.68,45.6
4,FF,2023-09-30 00:00:00,95.9,-2.10,5.51,"Smith, Drew",664761,622098,None,foul,...,1,4,3.0,2.0,NaN,1.0,0.90,0.78,0.78,39.6


In [4]:
df.columns

Index(['pitch_type', 'game_date', 'release_speed', 'release_pos_x',
       'release_pos_z', 'player_name', 'batter', 'pitcher', 'events',
       'description',
       ...
       'n_thruorder_pitcher', 'n_priorpa_thisgame_player_at_bat',
       'pitcher_days_since_prev_game', 'batter_days_since_prev_game',
       'pitcher_days_until_next_game', 'batter_days_until_next_game',
       'api_break_z_with_gravity', 'api_break_x_arm', 'api_break_x_batter_in',
       'arm_angle'],
      dtype='object', length=113)

In [5]:
df.pitch_type.unique()

array(['FF', 'SL', 'CH', 'CU', 'SI', 'FC', 'FS', 'ST', 'KC', 'SV', None,
       'PO', 'EP', 'FA', 'FO', 'KN', 'CS', 'SC'], dtype=object)

In [6]:
df.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,n_thruorder_pitcher,n_priorpa_thisgame_player_at_bat,pitcher_days_since_prev_game,batter_days_since_prev_game,pitcher_days_until_next_game,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle
0,FF,2023-09-30 00:00:00,96.1,-2.08,5.57,"Smith, Drew",664761,622098,field_out,hit_into_play,...,1,4,3.0,2.0,NaN,1.0,0.85,0.55,0.55,41.2
1,FF,2023-09-30 00:00:00,96.5,-1.94,5.59,"Smith, Drew",664761,622098,None,foul,...,1,4,3.0,2.0,NaN,1.0,0.94,0.70,0.70,39.9
2,SL,2023-09-30 00:00:00,86.6,-1.91,5.54,"Smith, Drew",664761,622098,None,ball,...,1,4,3.0,2.0,NaN,1.0,3.17,-0.48,-0.48,43.8
3,SL,2023-09-30 00:00:00,87.1,-2.01,5.55,"Smith, Drew",664761,622098,None,swinging_strike,...,1,4,3.0,2.0,NaN,1.0,2.98,-0.68,-0.68,45.6
4,FF,2023-09-30 00:00:00,95.9,-2.10,5.51,"Smith, Drew",664761,622098,None,foul,...,1,4,3.0,2.0,NaN,1.0,0.90,0.78,0.78,39.6


In [7]:
fields_to_keep = [
    'game_pk',
    'at_bat_number',
    'pitch_number',
    'pitch_type',
    'stand', 
    'p_throws',
    'batter',
    'pitcher', 
    'player_name',
    'zone',
    'balls',
    'strikes',
    'description',
    'estimated_ba_using_speedangle',
    'estimated_woba_using_speedangle',
    'launch_speed',
    'launch_angle',
    'inning',
    'on_3b',
    'on_2b',
    'on_1b',
    'fld_score',
    'bat_score',
]

#keep useful fields, fill N/A
demogs = df[fields_to_keep]
demogs['pitch_type'].fillna('N/A', inplace=True)
demogs.head()

/var/folders/2m/c68ytcbn1sbb3vqqhzqdy7rm0000gn/T/ipykernel_52365/4197880950.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demogs['pitch_type'].fillna('N/A', inplace=True)


,game_pk,at_bat_number,pitch_number,pitch_type,stand,p_throws,batter,pitcher,player_name,zone,...,estimated_ba_using_speedangle,estimated_woba_using_speedangle,launch_speed,launch_angle,inning,on_3b,on_2b,on_1b,fld_score,bat_score
0,716393,78,5,FF,R,R,664761,622098,"Smith, Drew",11.0,...,0.001,0.0,58.8,70.0,9,NaN,NaN,NaN,11,4
1,716393,78,4,FF,R,R,664761,622098,"Smith, Drew",5.0,...,NaN,NaN,76.4,24.0,9,NaN,NaN,NaN,11,4
2,716393,78,3,SL,R,R,664761,622098,"Smith, Drew",14.0,...,NaN,NaN,NaN,NaN,9,NaN,NaN,NaN,11,4
3,716393,78,2,SL,R,R,664761,622098,"Smith, Drew",14.0,...,NaN,NaN,NaN,NaN,9,NaN,NaN,NaN,11,4
4,716393,78,1,FF,R,R,664761,622098,"Smith, Drew",1.0,...,NaN,NaN,81.2,11.0,9,NaN,NaN,NaN,11,4


In [8]:
#Group pitch types in fastball or offspeed
def pitch_group(pitch):
    """
    Takes some string named pitch ('CH', 'FF', 'FC' etc) and checks whether it's
    a fastball (cutter, four-seam, two-seam, sinker, splitter)
    """
    if pitch == 'N/A':
        return 'N/A'
    elif pitch in ['FC', 'FF', 'FS', 'FT', 'SI']:
        return "FASTBALL"
    else:
        return "OFFSPEED"

In [9]:
demogs['pitch_group'] = demogs['pitch_type'].apply(pitch_group)

/var/folders/2m/c68ytcbn1sbb3vqqhzqdy7rm0000gn/T/ipykernel_52365/60364088.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demogs['pitch_group'] = demogs['pitch_type'].apply(pitch_group)


In [10]:
def count_type(strikes, balls):
    """
    Takes in # of strikes and balls and decides if the pitcher is ahead or behind in the count
    """
    if strikes == 2 or (strikes == 1 and balls == 0):
        return 'AHEAD'
        #Consdider all 2 strike counts (inclduing 3-2, 2-2) as a pitchers count
    elif balls > strikes:
        return "BEHIND"
    elif strikes == balls:
        return "EVEN"
    else:
        raise Exception('Missing Count Information')

In [11]:
demogs['count_type'] = demogs.apply(lambda x: count_type(x.strikes, x.balls), axis=1)
demogs.head()

/var/folders/2m/c68ytcbn1sbb3vqqhzqdy7rm0000gn/T/ipykernel_52365/2151467180.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demogs['count_type'] = demogs.apply(lambda x: count_type(x.strikes, x.balls), axis=1)


,game_pk,at_bat_number,pitch_number,pitch_type,stand,p_throws,batter,pitcher,player_name,zone,...,launch_speed,launch_angle,inning,on_3b,on_2b,on_1b,fld_score,bat_score,pitch_group,count_type
0,716393,78,5,FF,R,R,664761,622098,"Smith, Drew",11.0,...,58.8,70.0,9,NaN,NaN,NaN,11,4,FASTBALL,AHEAD
1,716393,78,4,FF,R,R,664761,622098,"Smith, Drew",5.0,...,76.4,24.0,9,NaN,NaN,NaN,11,4,FASTBALL,AHEAD
2,716393,78,3,SL,R,R,664761,622098,"Smith, Drew",14.0,...,NaN,NaN,9,NaN,NaN,NaN,11,4,OFFSPEED,AHEAD
3,716393,78,2,SL,R,R,664761,622098,"Smith, Drew",14.0,...,NaN,NaN,9,NaN,NaN,NaN,11,4,OFFSPEED,AHEAD
4,716393,78,1,FF,R,R,664761,622098,"Smith, Drew",1.0,...,81.2,11.0,9,NaN,NaN,NaN,11,4,FASTBALL,EVEN


In [12]:
#calculate score differential
demogs['score_diff'] = demogs['fld_score'] - demogs['bat_score']

/var/folders/2m/c68ytcbn1sbb3vqqhzqdy7rm0000gn/T/ipykernel_52365/3890893577.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demogs['score_diff'] = demogs['fld_score'] - demogs['bat_score']


In [13]:
#Get Batter ID to Batter Name map
player_id = playerid_reverse_lookup(demogs['batter'])
player_id['batter_name'] = player_id['name_last'].str.title() + ', ' + player_id['name_first'].str.title() 
player_id = player_id[['key_mlbam', 'batter_name']]
player_id.head()

Gathering player lookup table. This may take a moment.


,key_mlbam,batter_name
0,680911,"Miller, Owen"
1,669242,"Edman, Tommy"
2,660636,"Castillo, Diego"
3,676914,"Schneider, Davis"
4,670032,"Lopez, Nicky"


In [14]:
# Check for uniqueness
assert not player_id['key_mlbam'].duplicated().any()

In [15]:
demogs = pd.merge(demogs, player_id, how='left',left_on='batter', right_on='key_mlbam')
demogs.head()

,game_pk,at_bat_number,pitch_number,pitch_type,stand,p_throws,batter,pitcher,player_name,zone,...,on_3b,on_2b,on_1b,fld_score,bat_score,pitch_group,count_type,score_diff,key_mlbam,batter_name
0,716393,78,5,FF,R,R,664761,622098,"Smith, Drew",11.0,...,NaN,NaN,NaN,11,4,FASTBALL,AHEAD,7,664761.0,"Bohm, Alec"
1,716393,78,4,FF,R,R,664761,622098,"Smith, Drew",5.0,...,NaN,NaN,NaN,11,4,FASTBALL,AHEAD,7,664761.0,"Bohm, Alec"
2,716393,78,3,SL,R,R,664761,622098,"Smith, Drew",14.0,...,NaN,NaN,NaN,11,4,OFFSPEED,AHEAD,7,664761.0,"Bohm, Alec"
3,716393,78,2,SL,R,R,664761,622098,"Smith, Drew",14.0,...,NaN,NaN,NaN,11,4,OFFSPEED,AHEAD,7,664761.0,"Bohm, Alec"
4,716393,78,1,FF,R,R,664761,622098,"Smith, Drew",1.0,...,NaN,NaN,NaN,11,4,FASTBALL,EVEN,7,664761.0,"Bohm, Alec"


In [16]:
#Create boolean fields to indicate if there is a runner on a given base
demogs['on_1b_ind'] = pd.notnull(demogs['on_1b'])
demogs['on_2b_ind'] = pd.notnull(demogs['on_2b'])
demogs['on_3b_ind'] = pd.notnull(demogs['on_3b'])

In [17]:
demogs = demogs.astype({'pitch_number':'int'})
demogs = demogs.astype({'pitch_number':'str'})
demogs['pitch_num'] = 'pitch_' + demogs['pitch_number']
demogs = demogs.astype({'pitch_number':'int'})

In [20]:
seq_stacked = demogs[[
    'game_pk', 'at_bat_number', 'pitch_num', 'pitch_group',
]].drop_duplicates()
seq_stacked.head()

,game_pk,at_bat_number,pitch_num,pitch_group
0,716393,78,pitch_5,FASTBALL
1,716393,78,pitch_4,FASTBALL
2,716393,78,pitch_3,OFFSPEED
3,716393,78,pitch_2,OFFSPEED
4,716393,78,pitch_1,FASTBALL


In [21]:
# Create pitch sequences for each record
# Allows for analysis for pitch sequencing
seq = seq_stacked.set_index(['game_pk', 'at_bat_number', 'pitch_num']).unstack('pitch_num')['pitch_group'].reset_index()
seq.head()

pitch_num,game_pk,at_bat_number,pitch_1,pitch_10,pitch_11,pitch_12,pitch_13,pitch_14,pitch_15,pitch_16,pitch_2,pitch_3,pitch_4,pitch_5,pitch_6,pitch_7,pitch_8,pitch_9
0,716366,1,FASTBALL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,716366,2,FASTBALL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OFFSPEED,FASTBALL,FASTBALL,OFFSPEED,FASTBALL,NaN,NaN,NaN
2,716366,3,FASTBALL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FASTBALL,FASTBALL,OFFSPEED,FASTBALL,OFFSPEED,OFFSPEED,NaN,NaN
3,716366,4,FASTBALL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FASTBALL,FASTBALL,NaN,NaN,NaN,NaN,NaN,NaN
4,716366,5,FASTBALL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OFFSPEED,OFFSPEED,FASTBALL,FASTBALL,FASTBALL,NaN,NaN,NaN


In [22]:
full_data = pd.merge(demogs, seq, how='left', on=['game_pk', 'at_bat_number'])
full_data.head()

,game_pk,at_bat_number,pitch_number,pitch_type,stand,p_throws,batter,pitcher,player_name,zone,...,pitch_15,pitch_16,pitch_2,pitch_3,pitch_4,pitch_5,pitch_6,pitch_7,pitch_8,pitch_9
0,716393,78,5,FF,R,R,664761,622098,"Smith, Drew",11.0,...,NaN,NaN,OFFSPEED,OFFSPEED,FASTBALL,FASTBALL,NaN,NaN,NaN,NaN
1,716393,78,4,FF,R,R,664761,622098,"Smith, Drew",5.0,...,NaN,NaN,OFFSPEED,OFFSPEED,FASTBALL,FASTBALL,NaN,NaN,NaN,NaN
2,716393,78,3,SL,R,R,664761,622098,"Smith, Drew",14.0,...,NaN,NaN,OFFSPEED,OFFSPEED,FASTBALL,FASTBALL,NaN,NaN,NaN,NaN
3,716393,78,2,SL,R,R,664761,622098,"Smith, Drew",14.0,...,NaN,NaN,OFFSPEED,OFFSPEED,FASTBALL,FASTBALL,NaN,NaN,NaN,NaN
4,716393,78,1,FF,R,R,664761,622098,"Smith, Drew",1.0,...,NaN,NaN,OFFSPEED,OFFSPEED,FASTBALL,FASTBALL,NaN,NaN,NaN,NaN


In [23]:
# Clean data such that a we only show pitches so for in a given at bat. 
# Ex. If a given pitch is the 3rd pitch in at bat that last 5 pitches, only pitch_1, pitch_2, and pitch_3 will be populated
field_name = ''
for x in range(2, full_data['pitch_number'].max()+1):
    field_name = f'pitch_{str(x)}'
    full_data.loc[full_data['pitch_number'] <= x, field_name] = np.nan

In [24]:
# create offsets
# identifies pitch number in relation to the given pitch
# Ex. If we are examining the 3rd pitch in at bat, pitch_offset_1 is the 2nd pitch in that at bat, and pitch_offset_2 is the 1st pitch in that at bat.
field_name = ''
for x in range(1, full_data['pitch_number'].max()):
    field_name = 'pitch_offset_' + str(x)
    for y in range (x+1, full_data['pitch_number'].max()+1):
        offset_pitch = y - x
        pitch_name = f'pitch_{offset_pitch}'
        full_data.loc[full_data['pitch_number'] == y, field_name] = full_data[pitch_name]

In [25]:
full_data.head()

,game_pk,at_bat_number,pitch_number,pitch_type,stand,p_throws,batter,pitcher,player_name,zone,...,pitch_offset_6,pitch_offset_7,pitch_offset_8,pitch_offset_9,pitch_offset_10,pitch_offset_11,pitch_offset_12,pitch_offset_13,pitch_offset_14,pitch_offset_15
0,716393,78,5,FF,R,R,664761,622098,"Smith, Drew",11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,716393,78,4,FF,R,R,664761,622098,"Smith, Drew",5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,716393,78,3,SL,R,R,664761,622098,"Smith, Drew",14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,716393,78,2,SL,R,R,664761,622098,"Smith, Drew",14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,716393,78,1,FF,R,R,664761,622098,"Smith, Drew",1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
full_data.columns

Index(['game_pk', 'at_bat_number', 'pitch_number', 'pitch_type', 'stand',
       'p_throws', 'batter', 'pitcher', 'player_name', 'zone', 'balls',
       'strikes', 'description', 'estimated_ba_using_speedangle',
       'estimated_woba_using_speedangle', 'launch_speed', 'launch_angle',
       'inning', 'on_3b', 'on_2b', 'on_1b', 'fld_score', 'bat_score',
       'pitch_group', 'count_type', 'score_diff', 'key_mlbam', 'batter_name',
       'on_1b_ind', 'on_2b_ind', 'on_3b_ind', 'pitch_num', 'pitch_1',
       'pitch_10', 'pitch_11', 'pitch_12', 'pitch_13', 'pitch_14', 'pitch_15',
       'pitch_16', 'pitch_2', 'pitch_3', 'pitch_4', 'pitch_5', 'pitch_6',
       'pitch_7', 'pitch_8', 'pitch_9', 'pitch_offset_1', 'pitch_offset_2',
       'pitch_offset_3', 'pitch_offset_4', 'pitch_offset_5', 'pitch_offset_6',
       'pitch_offset_7', 'pitch_offset_8', 'pitch_offset_9', 'pitch_offset_10',
       'pitch_offset_11', 'pitch_offset_12', 'pitch_offset_13',
       'pitch_offset_14', 'pitch_offset_15']

In [27]:
full_data.rename(
    columns={
        "game_pk":"game_id",
        "stand":"batter_stand",
        "p_throws":"pitcher_throws"
    },
    inplace=True
)

In [28]:
full_data.drop(['on_3b', 'on_2b', 'on_1b', 'fld_score', 'bat_score'], axis=1, inplace=True)
full_data.head()

,game_id,at_bat_number,pitch_number,pitch_type,batter_stand,pitcher_throws,batter,pitcher,player_name,zone,...,pitch_offset_6,pitch_offset_7,pitch_offset_8,pitch_offset_9,pitch_offset_10,pitch_offset_11,pitch_offset_12,pitch_offset_13,pitch_offset_14,pitch_offset_15
0,716393,78,5,FF,R,R,664761,622098,"Smith, Drew",11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,716393,78,4,FF,R,R,664761,622098,"Smith, Drew",5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,716393,78,3,SL,R,R,664761,622098,"Smith, Drew",14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,716393,78,2,SL,R,R,664761,622098,"Smith, Drew",14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,716393,78,1,FF,R,R,664761,622098,"Smith, Drew",1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
full_data.to_sql('pitch_clean', conn, index=True, if_exists='replace')

743040

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=766ac258-4b25-4aeb-9ca3-1c0ba0991c26' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>